In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import multilabel_confusion_matrix as confusion_matrix

In [2]:
data = pd.read_csv('../data/train.csv')
train_set = data.sample(frac=0.9)
train_set_features = train_set.loc[:, train_set.columns != 'label']
validation_set = data.drop(train_set.index)
validation_set_features = validation_set.loc[:, validation_set.columns != 'label']

In [ ]:
# k from knn
k = 10
import time
start_time = time.perf_counter ()

# m train items
m = len(train_set)
train_labels = train_set[0:m]["label"]
train_a = train_set_features[0:m].to_numpy()

# b validation items
b = len(validation_set)
validation_labels = validation_set[0:b]["label"]
validation_a = validation_set_features[0:b].to_numpy()

# b x m matrix of distances from each validation item to each train item
distances = np.sqrt(np.sum(np.square(train_a - validation_a[:, None]), axis=2))


def get_label(column, labels):
    column.reset_index(drop=True, inplace=True)
    labels.reset_index(drop=True, inplace=True)
    merged = pd.concat([column.rename("distance"), labels], axis=1)
    return merged.nsmallest(k, columns=["distance"])["label"].mode()[0]


predictions = pd.DataFrame(distances.T).apply(lambda x: get_label(x, train_labels), axis=0)
end_time = time.perf_counter ()
print(end_time - start_time, "seconds")

In [ ]:
report = classification_report(validation_set['label'], predictions)
precision, recall, fscore, support = score(validation_set['label'], predictions, average='weighted')
print('Precision : {}'.format(precision))
print('Recall    : {}'.format(recall))
print('F-score   : {}'.format(fscore))

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=10, algorithm='kd_tree')
knn_model.fit(train_set_features, train_set.loc[:, train_set.columns == 'label'].values.flatten())
b = knn_model.predict(validation_set_features)

In [ ]:
reportb = classification_report(validation_set['label'], b)
precision, recall, fscore, support = score(validation_set['label'], b, average='weighted')
print('Precision : {}'.format(precision))
print('Recall    : {}'.format(recall))
print('F-score   : {}'.format(fscore))